# CSV Tables de produção por estágio e colheita

* Table 1 - Área de produção por estágio
    * codcult | safra | dataprev | %e0 | %e1 | ... | %e7 

* Table 2 - Área de produção de colheita
    *  codcult | safra | dataprev | %colhido 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from datetime import timedelta
from sklearn import preprocessing

clima_mga = pd.read_csv("dataset/clima_mga.csv")
clima_mga.data = clima_mga.data.astype('datetime64[ns]')

est = pd.read_excel("dataset/cocamar_lazaro.xlsx", "Estádio da Propriedade")
est.columns = est.columns.str.strip()
# Remoção da coluna codresp com valor constante
est = est.drop(["codresp"], axis=1)

In [2]:
df = est.groupby(["safra","codest","codcult","area","datprev"], as_index=None).agg({'prodest':'sum','prodobtida':'sum'})
df = df.drop(['prodest','prodobtida'],axis=1)

In [3]:
le = preprocessing.LabelEncoder()

df.safra = le.fit_transform(df.safra)
le.inverse_transform([0])

array(['2010/2010'], dtype=object)

## Drop culturas 6 e 12

* Cultura 6:
    * Safra 2010/2011 com estágios 7 e 8 
    * Safra 2012/2013 com estágios 6 e 8
* Cultura 12:
    * Apresenta 3 safras
    * Entretanto apresenta muitos registros repetidos de área de produção em datas diferentes 
    * Causa inconsistência na análise de áreas de colheita 

In [4]:
df = df[(df.codcult!=6)&(df.codcult!=12)]

## Guarda em dicionário, as maiores areas de cada safra para cada cultura

* Dicionário de culturas
* Para cada cultura:
    * Cria um dicionário Key = safra, Value = maior área e8 


## Verifica se as maiores áreas de cada safra correspondem à maior área geral (todos os estágios)

In [5]:
differ = []
cult_sem_e8 = []
area_e8 = {}

culturas = np.sort((df.codcult.unique()))
# Para cada cultura
for cult in culturas:
    aux = df[df.codcult == cult]
    safras = np.sort(aux.safra.unique())
    # Para cada safra
    sdict={}
    for saf in safras:
        # Busca pelo maior valor área em e8 e geral
        try:
            maior_e8 = max(aux[(aux.safra==saf)&(aux.codest==8)]["area"].to_list())
            maior_geral = max(aux[(aux.safra==saf)]["area"].to_list())
            sdict[saf] = max(maior_e8,maior_geral)
            if maior_e8 != maior_geral:
                differ.append([cult,saf])
        # Se não possui e8, salva info de cultura e safra
        except:
            cult_sem_e8.append([cult,saf])
        area_e8[str(cult)] = sdict

print("Área e8 e área geral diferentes: ", differ, "\n")
print("Culturas e safras sem informação de e8: ", cult_sem_e8, "\n")
print("Dicionário de culturas e maior área: \n", area_e8)

Área e8 e área geral diferentes:  [] 

Culturas e safras sem informação de e8:  [[120, 1], [120, 20]] 

Dicionário de culturas e maior área: 
 {'16': {1: 15860, 3: 15180, 5: 14524, 7: 14363, 9: 14263, 11: 12969, 13: 18402, 15: 12674, 17: 12577, 19: 9489}, '18': {0: 3285, 2: 2475, 4: 1455, 6: 1430, 8: 2003, 10: 2940, 12: 1872, 14: 1578, 16: 1140, 18: 1715}, '120': {2: 10031, 4: 10060, 6: 11050, 8: 11200, 10: 7900, 12: 10577, 14: 11293, 16: 10420, 18: 10062}}


## Elimina safras que não apresentam estágio 8

In [6]:
for elem in cult_sem_e8:
    df = df[~((df.codcult==elem[0])&(df.safra==elem[1]))]

## TABLE 2  - Porcentagem colhida
* codcult | safra | dataprev | %colhido

In [7]:
t2 = df[df.codest==8].reset_index()
t2 = t2.groupby(["codcult"], as_index=None).apply(lambda x: x.sort_values(["datprev"], ascending = True)).drop(["index"], axis=1)
t2.head(20)

safra  codest  codcult   area    datprev
0 9       1       8       16   1195 2011-02-25
  10      1       8       16   3710 2011-03-04
  11      1       8       16   8340 2011-03-11
  12      1       8       16  14524 2011-03-18
  13      1       8       16  15637 2011-03-25
  14      1       8       16  15810 2011-04-01
  15      1       8       16  15860 2011-04-08
  35      3       8       16     48 2012-02-10
  36      3       8       16    490 2012-02-17
  37      3       8       16   1931 2012-02-24
  38      3       8       16   4726 2012-03-02
  39      3       8       16  12403 2012-03-09
  40      3       8       16  14947 2012-03-16
  41      3       8       16  15086 2012-03-23
  42      3       8       16  15180 2012-03-30
  43      3       8       16  15180 2012-04-05
  44      3       8       16  15180 2012-04-13
  64      5       8       16    190 2013-02-01
  65      5       8       16    685 2013-02-08
  66      5       8       16    885 2013-02-15

### Cálculo % área colhida em cada data 

In [8]:
colhida = []
for ind, row in t2.iterrows():
    saf = row["safra"]
    cult = row["codcult"]
    area = row["area"]
    
    colhida.append(np.round((area/area_e8[str(cult)][saf])*100,1))
    #print(area, area_e8[str(cult)][saf])

### Adição de coluna % de colheita

In [9]:
t2["colheita(%)"]=colhida
t2.to_csv("dataset/table2-colheita.csv",index=None)
t2.head()

safra  codest  codcult   area    datprev  colheita(%)
0 9       1       8       16   1195 2011-02-25          7.5
  10      1       8       16   3710 2011-03-04         23.4
  11      1       8       16   8340 2011-03-11         52.6
  12      1       8       16  14524 2011-03-18         91.6
  13      1       8       16  15637 2011-03-25         98.6

## TABLE 1 - Porcentagem por estágio da planta
* codcult | safra | dataprev | %e0 | %e1 | ... | %e7

In [10]:
table = df.reset_index()
table = table.groupby(["codcult","datprev","codest","safra"], as_index=None).agg({'area':'sum'})
tab = {"codcult":[],"safra":[],"datprev":[],"area_total":[],"e1":[],"e2":[],"e3":[],"e4":[],"e5":[],"e6":[],"e7":[],"e8":[]}


In [11]:
# COM PORCENTAGEM

for dat in table.datprev.unique():
    
    # FILTRO POR DATA
    aux = table[table.datprev==dat]
    
    cult = []
    est = []
    for ind, row in aux.iterrows():
                
        safra =  row["safra"]
        cultura = row["codcult"]
        area = row["area"]
        estagio = row["codest"]
        safra_maior_area = area_e8[str(cultura)][safra]
        #print(area/safra_maior_area)
        
        # SE NOVA CULTURA, NOVA LINHA 
        if cultura not in cult:
            #print("new line ", cultura)
            
            # SE NÃO FOR A PRIMEIRA LINHA, PREENCHE ESTÁGIOS VAZIOS
            # DA LINHA ANTERIOR COM ZERO
            if(len(cult)>0):
                #print("preenche")
                for i in range(1,9):
                    if i not in est:
                        tab["e"+str(i)].append(0)  
                est = []
            
            # ADICIONA INFORMAÇOES INICIAIS
            tab["area_total"].append(safra_maior_area)
            tab["codcult"].append(cultura)
            tab["datprev"].append(dat)
            tab["safra"].append(safra)
            valor = np.round((area/safra_maior_area)*100,2)
            tab["e"+str(estagio)].append(valor)
 
            est.append(estagio)
            cult.append(cultura)
            #print(tab)
            
        # CULTURA JÁ EXISTE, ATUALIZA LINHA COM DADOS DE OUTRO ESTÁGIO
        else:
            #print("update")
            valor = np.round((area/safra_maior_area)*100,2)
            tab["e"+str(estagio)].append(valor)
            est.append(estagio)
            
    
    # FIM DE TABELA, PREENCHE ESTÁGIOS VAZIOS
    # DA ÚLTIMA LINHA COM ZERO
    for i in range(1,9):
        if i not in est:
            tab["e"+str(i)].append(0)
            
            
t1 = pd.DataFrame.from_dict(tab)
#t1.to_csv("dataset/table1-%estagios.csv",index=None)


## Tabela 1 com porcentagem de estágio acumulada  (T3)

In [12]:
def summ_est(vec):   
    
    aux = []
    for i in range(8):  #i=0
        
        summ = vec[i] # vetor e1 - e8
        for j in range(i+1,8): #j=1    
            summ += vec[j] 
        aux.append(summ)
    #print(aux)
    return pd.Series(aux)
        
t1[["e1","e2","e3","e4","e5","e6","e7","e8"]] = t1[["e1","e2","e3","e4","e5","e6","e7","e8"]].apply(summ_est,axis=1)
# Remoção de linhas onde a colheita já iniciou
# Conflito de datend maior que datprev
t1 = t1[(t1.e8 == 0.0)]
t1 = t1.drop(["area_total","e8"],axis=1)
t1.to_csv("dataset/table1-sem-clima.csv",index=None)
t1.head(20)

,codcult,safra,datprev,e1,e2,e3,e4,e5,e6,e7
0,16,1,2010-09-16,1.89,1.26,0.63,0.00,0.0,0.0,0.0
2,16,1,2010-10-08,0.13,0.00,0.00,0.00,0.0,0.0,0.0
3,16,1,2010-10-15,12.08,0.13,0.00,0.00,0.0,0.0,0.0
4,16,1,2010-10-22,47.40,12.08,0.13,0.00,0.0,0.0,0.0
5,16,1,2010-10-29,79.06,47.39,14.40,0.00,0.0,0.0,0.0
6,16,1,2010-11-05,97.17,79.07,47.40,0.00,0.0,0.0,0.0
7,16,1,2010-11-12,100.01,97.17,79.07,0.00,0.0,0.0,0.0
8,16,1,2010-11-19,100.00,100.00,97.16,0.00,0.0,0.0,0.0
9,16,1,2010-11-26,100.00,100.00,100.00,0.00,0.0,0.0,0.0
10,16,1,2010-12-03,100.00,100.00,100.00,0.00,0.0,0.0,0.0


## Tabela 1 acumulada com informações de clima 

In [13]:
def get_params(df, TOL, date, interval):
    a = date + timedelta(days = interval)
    if interval > 0:
        subset = df.loc[(df.data>=date) & (df.data<=a)]
    elif interval < 0:
        subset = df.loc[(df.data<=date) & (df.data>=a)]
    tmed    = subset.tmed.mean()
    tmax    = subset.tmax.mean()
    tmin    = subset.tmin.mean()
    ur      = subset.ur.mean()
    nprec   = subset.loc[subset.precipitacao>TOL].precipitacao.count()*100/interval
    sprec   = subset.precipitacao.sum()
    dic = {'tmed': tmed, 'tmax': tmax, 'tmin': tmin, 'ur': ur, 'nprec': nprec, 'sprec': sprec}
    return pd.Series(dic)

#t1 = clima_mga.copy()
t1.rename(columns={'data':'datprev'}, inplace=True)

ts_precip = 10

t1[['n7tmed', 'p7tmax', 'p7tmin', 'p7ur', 'p7nprec', 'p7sprec']]      = t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, -7))
t1[['p7tmed', 'n7tmax', 'n7tmin', 'n7ur', 'n7nprec', 'n7sprec']]      = t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, 7))
t1[['p14tmed', 'p14tmax', 'p14tmin', 'p14ur', 'p14nprec', 'p14sprec']]= t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, -14))
t1[['n14tmed', 'n14tmax', 'n14tmin', 'n14ur', 'n14nprec', 'n14sprec']]= t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, 14))
t1[['p28tmed', 'p28tmax', 'p28tmin', 'p28ur', 'p28nprec', 'p28sprec']]= t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, -28))
t1[['n28tmed', 'n28tmax', 'n28tmin', 'n28ur', 'n28nprec', 'n28sprec']]= t1.datprev.apply(lambda data: get_params(clima_mga, ts_precip, data, 28))

In [14]:
t1.to_csv("dataset/table1-com-clima.csv",index=None)